In [1]:
import numpy as np
%matplotlib inline
import matplotlib as plt
import pandas as pd
import datetime as dt
import statsmodels.formula.api as smf
import scipy
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
df1 = pd.read_csv("/gws/open/Student/citibike/201610-citibike-tripdata.zip")
df = pd.read_csv("/gws/open/Student/citibike/201707-citibike-tripdata.csv.zip")

In [3]:
df1.columns = df.columns
data = pd.concat([df, df1])
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2


# Day or night trips affect the trip duration

## This is the first part of the assignment where we should test if the tripduration of a day trip is different to the trip duration of a night trip.

### H0: The duration of a trip made during the night is statistically not different from the duration of a trip done during the day.

### H1 The duration of a trip made during the night is significantly different (95% confidence level) to the duration of a trip done during the day.

In [ ]:
data['time']= data['starttime'].apply(lambda x : pd.to_datetime(x).time())

In [ ]:
data.head()

In [ ]:
nightstart = dt.datetime.strptime('19:00:00', '%H:%M:%S').time()
daystart = dt.datetime.strptime('07:00:00', '%H:%M:%S').time()

In [ ]:
print nightstart
print daystart

In [ ]:
def dayornight(t):
    if t > nightstart:
        return 'night'
    if t < daystart:
        return 'night'
    else:
        return 'day'

In [ ]:
data['dayornight'] = data['time'].apply(lambda x: dayornight(x))

In [ ]:
data.head()

In [ ]:
data.dayornight.value_counts().plot(kind='bar')

In [ ]:
df2 = data[data["dayornight"]=="day"]
df3 = data[data["dayornight"]=="night"]

In [ ]:
df2.describe()

In [ ]:
df3.describe()

In [ ]:
data.dayornight = data.dayornight.replace('day', 0)
data.dayornight = data.dayornight.replace('night', 1)

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
lm=smf.ols('tripduration~dayornight',data=data).fit()
lm.summary()

In [ ]:
scipy.stats.pearsonr(data["tripduration"], data ["dayornight"])

In [ ]:
scipy.stats.spearmanr(data["tripduration"], data ["dayornight"])

In [ ]:
#This graph is in order to see how is the sample distributed for the ks test. We will say Poisson.
df5 = data[data["tripduration"]<5000]
hist(df5.tripduration, bins = 40, color = "Grey", label = "Total trips");

In [ ]:
#Here we test that the distribution could be treated as a poisson.
scipy.stats.kstest(data["tripduration"], poisson, N=3309471)

In [ ]:
scipy.stats.ks_2samp(data[(data["dayornight"] == 0)]["tripduration"],data[(data["dayornight"] == 1)]["tripduration"])

## Test results.

### Pearson
If we are rejecting if the test result is < 0.05 then the Null Hypothesis is rejected and it is possible to say that nighttrips have a significantly different tripduration compared to daytrips. Although you can also see that the correlation is negative but very small, which means that night trips are shorter, but you can barely explain the duration of the trip by identifiying if it is a day or a night trip.

### Spearman
With a P value that is near to 0, we can reject the Null Hypothesis and as such, it is possible to say that nighttrips have a significantly different tripduration compared to daytrips. Although you can also see that the correlation is negative but very small, which means that night trips are shorter, but you can barely explain the duration of the trip by identifiying if it is a day or a night trip.


### K-S
With a P value that is near to 0, we can reject the Null Hypothesis and as such, it is possible to say that nighttrips have a significantly different tripduration compared to daytrips. Although you can also see that the correlation is very small, which means that night trips are shorter, but you can barely explain the duration of the trip by identifiying if it is a day or a night trip.

# Age of bikers is different for trips originating in Manhattan and in Brooklyn.

## This is the seconf part of the assignment where we should test if the average age of bikers is different depending on the place they take the bike.

### H0: The average age of people that take a citibike in Brooklyn statistically not different from the average age of people that take a citibike in Manhattan.

### H1 The average age of people that take a citibike in Brooklyn is significantly different (95% confidence level) to the average age of people that take a citibike in Manhattan.

In [4]:
data['age'] = 2017 - data['birth year']

In [5]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,age
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,31.0
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,36.0
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,33.0
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,32.0
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,35.0


In [6]:
def borough(xy):
    x,y = xy
    if (x>40.698215)&(y<-73.970755):
        return "Manhattan"        
    else:
        return "Brooklyn"
    

In [7]:
data["borough"] = data[["start station latitude","start station longitude"]].apply(borough, axis=1)

# The data is contaminated WARNING.

### Given that reverse geopy didn't allowed enough consultations, then it was necessary to devide brooklyn stations from Manhattan stations using a determined coordinate. In this case manhattan includes al New Jersey stations, whil brooklyn includes all Queens staitions. I is also true that some north eastern Manhattan stations started making part of Brooklyn.

In [8]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,age,borough
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,31.0,Brooklyn
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,36.0,Manhattan
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,33.0,Brooklyn
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,32.0,Brooklyn
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,35.0,Brooklyn


In [9]:
data.borough = data.borough.replace('Manhattan', 0)
data.borough = data.borough.replace('Brooklyn', 1)

In [10]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,age,borough
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,31.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,36.0,0
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,33.0,1
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,32.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,35.0,1


In [11]:
model1 = smf.ols(formula = "age ~ borough", data = data).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     4017.
Date:                Tue, 10 Oct 2017   Prob (F-statistic):               0.00
Time:                        23:38:40   Log-Likelihood:            -1.1285e+07
No. Observations:             2913427   AIC:                         2.257e+07
Df Residuals:                 2913425   BIC:                         2.257e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     38.2993      0.008   4836.004      0.000        38.284    38.315
borough       -0.9871      0.016    -63.377      0.000        -1.018    -0.957
==============================================================================
Omnibus:                   273162.887   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           359833.927
Skew:                           0.808   Prob(JB):                         0.00
Kurtosis:                       3.596   Cond. No.                         2.47
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
scipy.stats.pearsonr(data["age"], data ["borough"])

(nan, 1.0)

In [13]:
scipy.stats.spearmanr(data["age"], data ["borough"])

SpearmanrResult(correlation=-0.0010747605390165431, pvalue=0.050559587993928026)

In [15]:
scipy.stats.ks_2samp(data[(data["borough"] == 0)]["age"],data[(data["borough"] == 1)]["age"])

ValueError: ks_2samp only accepts non-nan inputs

# Test results.

### Pearson:

Pearson test is not usable in this case.

### Spearman:

Spearman test suggests that the Null Hypothesis cannot be rejected at a 95% confidence level, although it suggests a negative coefficient, which would mean that on average Brooklyn citibike users are younger

### K-S

K-S test is not working